In [50]:
#import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
#from sqlalchemy import func
import datetime
import sqlite3
import time
from polygon import RESTClient
from pymongo import MongoClient  # Added import for MongoDB
import csv

In [40]:
client = RESTClient("beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq")

curr_pair="EURUSDAUDUSDINRUSDJPYUSD"
window_size=5
duration = 40
duration = duration
measure1 = time.time()
measure2 = time.time()
count = 1

Base = declarative_base()

In [41]:
def check_and_drop_database(database_name):
        try:
            client = MongoClient(uri)
            # List all available databases
            database_list = client.list_database_names()

            if database_name in database_list:
                client.drop_database(database_name)
                print(f"Database '{database_name}' dropped successfully.")
            else:
                print(f"Database '{database_name}' does not exist.")
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            client.close()

def calculate_and_store_keltner_fd(mean_old,vol_old):
    mean_new = []
    vol_new = []
    # Connect to MongoDB
    mongo_client = MongoClient('mongodb://localhost:27017')
    
    
    db = mongo_client['forex_data_db']  # MongoDB database name
    collection = db['forex_data_collection']  #MongoDB collection name

    def calculate_keltner_channel(mean_value, vol, n):
        upper_band = mean_value + n * 0.025 * vol
        lower_band = mean_value - n * 0.025 * vol
        return upper_band, lower_band

    # Connect to the SQLite database
    conn = sqlite3.connect('forex_data.db')
    cursor = conn.cursor()
    currency_pairs = ['EURUSD', 'AUDUSD', 'INRUSD', 'JPYUSD']
    # Specify the columns you want to extract
    columns = []
    for pair in currency_pairs:
        columns.append( f'Converted_{pair}')


    # Construct the SQL query to select the specified columns
    select_query = f"SELECT {', '.join(columns)} FROM forex_data"

    # Execute the query
    cursor.execute(select_query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Calculate mean, max, and min for each column
    for i, column in enumerate(columns):
        # Extract the values for the current column
        values = [row[i] for row in rows]

        # Calculate mean, max, and min
        mean_value = sum(values) / len(values)
        max_value = max(values)
        min_value = min(values)
        vol = max_value - min_value

        # Calculate Keltner Channel Upper/Lower Bands for n from 1 to 100
        keltner_channel_results = {}
        for n in range(1, 101):
            upper_band, lower_band = calculate_keltner_channel(mean_value, vol, n)
            keltner_channel_results[str(n)] = {'Upper Band': upper_band, 'Lower Band': lower_band}

        # Calculate the number of times the price crosses the Keltner Channel
        crosses_count = 0
        for value in values:
            for n in range(1, 101):
                upper_band, lower_band = calculate_keltner_channel(mean_value, vol, n)
                if lower_band < value < upper_band:
                    crosses_count += 1
                    break

        # Calculate Fractal Dimension (FD)
        fd = 0 if vol == 0 else crosses_count / vol
        current_timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # Store the results in MongoDB
        document = {
            'Timestamp': current_timestamp,
            'Column': column,
            'Mean': mean_value,
            'Max': max_value,
            'Min': min_value,
            'Vol': vol,
            'FD': fd,
            
        }
        collection.insert_one(document)
        mean_new.append(mean_value)
        vol_new.append(vol)

        
    # Close the connections
    conn.close()
    mongo_client.close()
    # print(mean_new,vol_new)
    return mean_new,vol_new


In [42]:
class ForexData(Base):
    __tablename__ = 'forex_data'

    id = Column(Integer, primary_key=True)
    EURUSD = Column(String)
    Date_1 = Column(DateTime, default=datetime.datetime.now)
    Converted_EURUSD = Column(Float)
    Time_1 = Column(String)

    AUDUSD = Column(String)
    Date_2 = Column(DateTime, default=datetime.datetime.now)
    Converted_AUDUSD = Column(Float)
    Time_2 = Column(String)

    INRUSD = Column(String)
    Date_3 = Column(DateTime, default=datetime.datetime.now)
    Converted_INRUSD = Column(Float)
    Time_3 = Column(String)

    JPYUSD = Column(String)
    Date_4 = Column(DateTime, default=datetime.datetime.now)
    Converted_JPYUSD = Column(Float)
    Time_4 = Column(String)


In [43]:
engine = create_engine('sqlite:///forex_data.db')
Base.metadata.create_all(engine)
mean = [0,0,0,0]
volatility = [0,0,0,0]
Session = sessionmaker(bind=engine)
session = Session()

In [44]:

while count < int(duration) + 1:
    count_1=1
    if count_1<count:
        mean,volatility = calculate_and_store_keltner_fd(mean, volatility)
        engine.execute('DROP TABLE IF EXISTS forex_data')
        Base.metadata.create_all(engine)
   # else:
        #mean,volatility = calculate_and_store_keltner_fd(mean, volatility)
        
        
       
    while count_1 < int(window_size) + 1:
        if measure2 - measure1 >= 1.00:
            measure1 = measure2
            measure2 = time.time()
            res1 = client.get_real_time_currency_conversion(curr_pair[:3], curr_pair[3:6], precision=5)
            res2 = client.get_real_time_currency_conversion(curr_pair[6:9], curr_pair[9:12], precision=5)
            res3 = client.get_real_time_currency_conversion(curr_pair[12:15], curr_pair[15:18], precision=5)
            res4 = client.get_real_time_currency_conversion(curr_pair[18:21], curr_pair[21:24], precision=5)
            real_time_input = [curr_pair[:6], datetime.date.fromtimestamp(res1.last.timestamp/1000), res1.converted, datetime.datetime.fromtimestamp(res1.last.timestamp/1000).strftime('%H:%M:%S'),
                               curr_pair[6:12], datetime.date.fromtimestamp(res2.last.timestamp/1000), res2.converted, datetime.datetime.fromtimestamp(res2.last.timestamp/1000).strftime('%H:%M:%S'),
                               curr_pair[12:18], datetime.date.fromtimestamp(res3.last.timestamp/1000), res3.converted, datetime.datetime.fromtimestamp(res3.last.timestamp/1000).strftime('%H:%M:%S'),
                               curr_pair[18:24], datetime.date.fromtimestamp(res4.last.timestamp/1000), res4.converted, datetime.datetime.fromtimestamp(res4.last.timestamp/1000).strftime('%H:%M:%S')] 
            count_1 += 1
            count   += 1
            forex_data = ForexData(
            EURUSD=real_time_input[0],
            Converted_EURUSD=real_time_input[2],
            Time_1=real_time_input[3],
            AUDUSD=real_time_input[4],
            Converted_AUDUSD=real_time_input[6],
            Time_2=real_time_input[7],
            INRUSD=real_time_input[8],
            Converted_INRUSD=real_time_input[10],
            Time_3=real_time_input[11],
            JPYUSD=real_time_input[12],
            Converted_JPYUSD=real_time_input[14],
            Time_4=real_time_input[15])
            print("time elasepsed since last storage{f}",count_1)
            
            session.add(forex_data)
            session.commit()
            
        else:
            measure2 = time.time()


time elasepsed since last storage{f} 2
time elasepsed since last storage{f} 3
time elasepsed since last storage{f} 4
time elasepsed since last storage{f} 5
time elasepsed since last storage{f} 6
time elasepsed since last storage{f} 2
time elasepsed since last storage{f} 3
time elasepsed since last storage{f} 4
time elasepsed since last storage{f} 5
time elasepsed since last storage{f} 6
time elasepsed since last storage{f} 2
time elasepsed since last storage{f} 3
time elasepsed since last storage{f} 4
time elasepsed since last storage{f} 5
time elasepsed since last storage{f} 6
time elasepsed since last storage{f} 2
time elasepsed since last storage{f} 3
time elasepsed since last storage{f} 4
time elasepsed since last storage{f} 5
time elasepsed since last storage{f} 6
time elasepsed since last storage{f} 2
time elasepsed since last storage{f} 3
time elasepsed since last storage{f} 4
time elasepsed since last storage{f} 5
time elasepsed since last storage{f} 6
time elasepsed since last

In [51]:
from pymongo import MongoClient

# MongoDB connection URI
uri = 'mongodb://localhost:27017/'

def find_min_max_fd():
    try:
        client = MongoClient(uri)
        db = client['forex_data_db']  # MongoDB database name
        collection = db['forex_data_collection']  # MongoDB collection name

        # Aggregation pipeline to find min and max values of 'FD' field
        pipeline = [
            {"$group": {
                "_id": None,
                "min_fd": {"$min": "$FD"},
                "max_fd": {"$max": "$FD"}
            }}
        ]

        # Execute aggregation pipeline
        result = list(collection.aggregate(pipeline))

        # Extract min and max values
        min_fd = result[0]['min_fd']
        max_fd = result[0]['max_fd']

        return min_fd, max_fd

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        client.close()

def normalize_fd():
    try:
        min_fd, max_fd = find_min_max_fd()  # Get min and max values

        client = MongoClient(uri)
        db = client['forex_data_db']  # MongoDB database name
        collection = db['forex_data_collection']  # MongoDB collection name

        # Fetch all documents from the collection
        cursor = collection.find({})

        # Process each document and normalize 'FD', adding 'normalized_fd'
        for document in cursor:
            fd_value = document.get('FD')  # Get the 'FD' value from the document

            # Perform normalization using the retrieved min and max values
            normalized_value = (fd_value - min_fd) / (max_fd - min_fd)  # Normalization formula

            # Update the document to include the normalized value
            collection.update_one(
                {'_id': document['_id']},
                {'$set': {'normalized_fd': normalized_value}}  # Update 'normalized_fd' field
            )
        print("Normalization completed.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        client.close()

# Call the function to normalize the 'FD' field and add 'normalized_fd'
normalize_fd()



An error occurred: division by zero


In [52]:

def export_collection_to_csv():
    try:
        client = MongoClient(uri)
        db = client['forex_data_db']  # MongoDB database name
        collection = db['forex_data_collection']  # MongoDB collection name

        # Fetch all documents from the collection
        cursor = collection.find({})

        # CSV file path to save the data
        csv_file = 'C:/Users/aksha/Downloads/data_eng_fin/forex_data.csv'

        # Open the CSV file in write mode
        with open(csv_file, 'w', newline='') as csvfile:
            csv_writer = csv.DictWriter(csvfile, fieldnames=cursor[0].keys())

            # Write headers to the CSV file
            csv_writer.writeheader()

            # Write each document's data to the CSV file
            for document in cursor:
                csv_writer.writerow(document)

        print(f"Collection data exported to '{csv_file}'.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        client.close()

# Call the function to export collection data to a CSV file
export_collection_to_csv()

Collection data exported to 'C:/Users/aksha/Downloads/data_eng_fin/forex_data.csv'.
